In [2]:
import requests
import csv
import pandas as pd
import openpyxl as pxl

In [3]:
URL = 'https://opendata.maryland.gov/api/views/jwfa-fdxs/rows.csv?accessType=DOWNLOAD'


with requests.Session() as s:
    download = s.get(URL)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
MD = pd.DataFrame(columns = my_list[0]) 
for i in my_list[1:]:
    MD.loc[len(MD.index)] = i

In [4]:
URL = 'https://data.ny.gov/api/views/34dd-6g2j/rows.csv?accessType=DOWNLOAD'


with requests.Session() as s:
    download = s.get(URL)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
NY = pd.DataFrame(columns = my_list[0]) 
for i in my_list[1:]:
    NY.loc[len(NY.index)] = i

In [202]:
MD_temp
#MD.to_csv('MD.csv')

In [210]:
#MD_temp = MD.query("YEAR > 1989")
MD_temp = MD
MD_temp[['POPULATION', 'MURDER', 'RAPE', 'ROBBERY', 'AGG. ASSAULT', 'B & E', 'LARCENY THEFT', 'M/V THEFT']] = MD_temp[['POPULATION', 'MURDER', 'RAPE', 'ROBBERY', 'AGG. ASSAULT', 'B & E', 'LARCENY THEFT', 'M/V THEFT']].astype('int64')
MD_temp = MD_temp.loc[(MD['YEAR'] > 1989)]
MD_totals = MD_temp.groupby('JURISDICTION')[['POPULATION', 'MURDEwR', 'RAPE', 'ROBBERY', 'AGG. ASSAULT', 'B & E', 'LARCENY THEFT', 'M/V THEFT']].sum()
MD_totals['POPULATION'] = (MD_totals['POPULATION'] / 30).round(0)
MD_totals.rename(columns = {'POPULATION':'AVERAGE POPULATION'}, inplace = True)
MD_totals['TOTAL VIOLENT'] = MD_totals[['MURDER', 'RAPE', 'ROBBERY', 'AGG. ASSAULT']].sum(axis='columns')
MD_totals['TOTAL PROPERTY'] = MD_totals[['B & E', 'LARCENY THEFT', 'M/V THEFT']].sum(axis='columns')
MD_totals['TOTAL INDEX CRIMES'] = MD_totals[['MURDER', 'RAPE', 'ROBBERY', 'AGG. ASSAULT', 'B & E', 'LARCENY THEFT', 'M/V THEFT']].sum(axis='columns')
MD_totals.to_csv('MD_totals.csv')
MD_totals.head()

,AVERAGE POPULATION,MURDER,RAPE,ROBBERY,AGG. ASSAULT,B & E,LARCENY THEFT,M/V THEFT,TOTAL VIOLENT,TOTAL PROPERTY,TOTAL INDEX CRIMES
JURISDICTION,,,,,,,,,,,
Allegany County,76235.0,53,636,1004,7088,14665,50109,2175,8781,66949,75730
Anne Arundel County,525965.0,513,3620,20443,54576,92606,376912,41407,79152,510925,590077
Baltimore City,679782.0,8899,11535,194512,209994,313964,822257,215880,424940,1352101,1777041
Baltimore County,800368.0,971,7220,53795,116294,164945,628489,99601,178280,893035,1071315
Calvert County,81447.0,48,379,584,5498,9746,30597,2124,6509,42467,48976


In [36]:
x = 0
MD_condensed = pd.DataFrame()
for index, row in MD.iterrows():
    if int(row[1]) >= 1990:
        MD_condensed.at[x, 'County'] = row[0]
        MD_condensed.at[x, 'Year'] = row[1]
        MD_condensed.at[x, 'Population'] = int(row[2])
        MD_condensed.at[x, 'Index Count'] = int(row[3]) + int(row[4]) + int(row[5]) + int(row[6]) + int(row[7]) + int(row[8]) + int(row[9])
        MD_condensed.at[x, 'Index Rate'] = float(row[2])/(int(row[3]) + int(row[4]) + int(row[5]) + int(row[6]) + int(row[7]) + int(row[8]) + int(row[9]))
        MD_condensed.at[x, 'Violent Count'] = int(row[3]) + int(row[4]) + int(row[5]) + int(row[6])
        MD_condensed.at[x, 'Violent Rate'] =  float(row[2])/(int(row[3]) + int(row[4]) + int(row[5]) + int(row[6]))
        MD_condensed.at[x, 'Property Count'] = int(row[7]) + int(row[8]) + int(row[9])
        MD_condensed.at[x, 'Property Rate'] = float(row[2])/(int(row[7]) + int(row[8]) + int(row[9]))
        x += 1

In [117]:
MD_condensed.to_csv('MD_condensed.csv')
MD_condensed.head()

,County,Year,Population,Index Count,Index Rate,Violent Count,Violent Rate,Property Count,Property Rate
0,Allegany County,1990,74946.0,2388.0,31.384422,238.0,314.899160,2150.0,34.858605
1,Allegany County,1991,76175.0,2469.0,30.852572,318.0,239.544025,2151.0,35.413761
2,Allegany County,1992,76925.0,2691.0,28.586027,362.0,212.500000,2329.0,33.029197
3,Allegany County,1993,75274.0,2457.0,30.636549,380.0,198.089474,2077.0,36.241695
4,Allegany County,1994,75891.0,2434.0,31.179540,335.0,226.540299,2099.0,36.155788


In [211]:
NY.to_csv('NY.csv')
NY.head()

In [3]:
wb = pxl.load_workbook('Total_Index_Crime_by_County.xlsx', read_only=True, data_only=True)
col_names = ['County', 'Population', 'Murder', 'Forcible Sex Offenses', 'Robbery', 'Aggravated Assault','Burglary', 'Larceny', 'Motor Vehicle Theft', 'Total Index Crimes', '% Index change', 'Index Rate per 100,000', 'Rate Change', 'Year']
FL = pd.DataFrame()
for ws in wb:
    temp_df = pd.DataFrame(ws.values)
    FL = pd.concat([FL, temp_df], ignore_index=True)
FL.drop(14, axis=1, inplace=True)
FL.columns = col_names 

C:\Users\Gohan\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [47]:
FL = pd.read_csv("Florida_Data_Clean.csv")
FL.fillna(0, inplace=True)
FL.head()

,County,Population,Murder,Forcible Sex Offenses,Robbery,Aggravated Assault,Burglary,Larceny,Motor Vehicle Theft,Total Index Crimes,% Index change,"Index Rate per 100,000",Rate Change,Year
0,Alachua County,271588.0,16.0,236.0,350.0,1423.0,853.0,5121.0,715.0,8714.0,-3.285238624,3208.53646,-4.810094686,2020.0
1,Baker County,28532.0,2.0,11.0,3.0,116.0,33.0,212.0,29.0,406.0,2.525252525,1422.96369,1.508336555,2020.0
2,Bay County,174410.0,11.0,93.0,67.0,672.0,771.0,3451.0,381.0,5446.0,-16.63860401,3122.527378,-20.04504096,2020.0
3,Bradford County,28725.0,2.0,8.0,13.0,75.0,41.0,203.0,29.0,371.0,-33.15315315,1291.557876,-33.2532198,2020.0
4,Brevard County,606671.0,30.0,225.0,298.0,1735.0,1629.0,8161.0,872.0,12950.0,-10.64651901,2134.600137,-12.44368943,2020.0


In [208]:
x = 0
FL_condensed = pd.DataFrame()
for index, row in FL.iterrows():
    if int(row[13]) >= 1990:
        if(0 not in set([row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9]])):
            FL_condensed.at[x, 'County'] = row[0]
            FL_condensed.at[x, 'Year'] = row[13]
            FL_condensed.at[x, 'Population'] = int(row[1])
            FL_condensed.at[x, 'Index Count'] = row[9]
            FL_condensed.at[x, 'Index Rate'] = float(row[1])/(int(row[9]))
            FL_condensed.at[x, 'Violent Count'] = int(row[2]) + int(row[3]) + int(row[5])
            FL_condensed.at[x, 'Violent Rate'] =  float(row[1])/(int(row[2]) + int(row[3]) + int(row[5]))
            FL_condensed.at[x, 'Property Count'] = int(row[4]) + int(row[6]) + int(row[7]) + int(row[8]) 
            FL_condensed.at[x, 'Property Rate'] = float(row[1])/(int(row[4]) + int(row[6]) + int(row[7]) + int(row[8]))
            x += 1
        else:
            FL_condensed.at[x, 'County'] = row[0]
            FL_condensed.at[x, 'Year'] = row[13]
            x += 1
FL_condensed.dropna(subset=['Population'], inplace=True)
FL_condensed.to_csv('FL_condensed.csv')

In [206]:
FL_temp = FL.query("Year > 1989")
FL_totals = FL_temp.groupby('County')[['Population', 'Murder', 'Forcible Sex Offenses', 'Robbery', 'Aggravated Assault', 'Burglary', 'Larceny', 'Motor Vehicle Theft', 'Total Index Crimes']].sum()
FL_totals['Population'] = (FL_totals['Population'] / 30).round(0)
FL_totals.rename(columns = {'Population':'Average Population'}, inplace = True)
FL_totals['Total Violent Crimes'] = FL_totals[['Murder', 'Forcible Sex Offenses', 'Aggravated Assault']].sum(axis='columns')
FL_totals['Total Property Crimes'] = FL_totals[['Robbery', 'Burglary', 'Larceny', 'Motor Vehicle Theft']].sum(axis='columns')
FL_totals.head()

,Average Population,Murder,Forcible Sex Offenses,Robbery,Aggravated Assault,Burglary,Larceny,Motor Vehicle Theft,Total Index Crimes,Total Violent Crimes,Total Property Crimes
County,,,,,,,,,,,
Alachua County,239397.0,300.0,6499.0,12067.0,43131.0,85041.0,248831.0,25100.0,420969.0,49930.0,371039.0
Baker County,24771.0,33.0,207.0,292.0,2483.0,3312.0,8686.0,916.0,15929.0,2723.0,13206.0
Bay County,163000.0,268.0,3102.0,4413.0,21039.0,47698.0,166553.0,12245.0,255318.0,24409.0,230909.0
Bradford County,27597.0,35.0,381.0,415.0,3556.0,6618.0,14365.0,1464.0,26834.0,3972.0,22862.0
Brevard County,528842.0,649.0,8979.0,15989.0,70789.0,134764.0,398802.0,36055.0,666027.0,80417.0,585610.0


In [207]:
FL_totals.to_csv('FL_totals.csv')
#FL.to_csv('Florida_Data_Clean.csv', encoding='utf-8')
#FL.to_json('Florida_Data_Clean.json')

In [130]:
FL_temp.loc[FL_temp['County'] == 'Alachua County']

,County,Population,Murder,Forcible Sex Offenses,Robbery,Aggravated Assault,Burglary,Larceny,Motor Vehicle Theft,Total Index Crimes,% Index change,"Index Rate per 100,000",Rate Change,Year
0,Alachua County,271588.0,16.0,236.0,350.0,1423.0,853.0,5121.0,715.0,8714.0,-3.285238624,3208.53646,-4.810094686,2020.0
67,Alachua County,267306.0,6.0,282.0,293.0,1184.0,1008.0,5643.0,594.0,9010.0,-7.665505226,3370.668822,-9.052391404,2019.0
134,Alachua County,263291.0,14.0,285.0,337.0,1175.0,1144.0,6170.0,633.0,9758.0,7.942477876,3706.165422,6.594483196,2018.0
201,Alachua County,260003.0,9.0,249.0,290.0,1097.0,1120.0,5751.0,524.0,9040.0,-1.845819761,3476.882959,-2.956081735,2017.0
268,Alachua County,257062.0,4.0,216.0,271.0,966.0,1256.0,5963.0,534.0,9210.0,3.879990977,3582.793256,3.003487641,2016.0
335,Alachua County,254893.0,3.0,183.0,312.0,999.0,1236.0,5722.0,411.0,8866.0,-1.978993919,3478.322276,-3.579906649,2015.0
402,Alachua County,250730.0,3.0,164.0,236.0,1003.0,1339.0,5879.0,421.0,9045.0,-1.961846954,3607.466199,-3.028524582,2014.0
469,Alachua County,248002.0,9.0,114.0,236.0,1023.0,1486.0,5996.0,362.0,9226.0,-8.977900552,3720.131289,-9.430071206,2013.0
536,Alachua County,246770.0,10.0,168.0,253.0,1057.0,1919.0,6372.0,357.0,10136.0,-3.300896775,4107.468493,-3.07871259,2012.0
603,Alachua County,247337.0,12.0,156.0,281.0,1090.0,2023.0,6485.0,435.0,10482.0,-2.347680268,4237.942564,0.950997776,2011.0
